# 불량 항목별 재현율 분석
- 작성자 : 강성현
- 대상  : 구미사업장

## 데이터 불러오기

In [1]:
# # !pip config set global.index-url 'https://meta1jsw.id:AKCp8nys2TLM1u1gyPoYikXU8m5uBgPbFAZgNyxXdHtLssf7H8hZxjLjyzZ8trQR8sxaFmvcG@bart.sec.samsung.net/artifactory/api/pypi/swgpypi-pypi/simple'
# !pip config set global.index-url 'https://bart.sec.samsung.net/artifactory/api/pypi/swgpypi-pypi/simple'
# !pip config set global.trusted-host 'bart.sec.samsung.net'

# !sudo echo "10.42.107.50 bart.sec.samsung.net" | sudo tee -a /etc/hosts
# !sudo echo "107.118.213.115 dt-da-rest-sehc-dev.secmessehc.sdspaas.io dt-vd-rest-sehc-dev.secmessehc.sdspaas.io" | sudo tee -a /etc/hosts

# !pip install tomli plotly loguru
# !pip install --upgrade typing_extensions

In [2]:
!python --version

Python 3.9.7


In [3]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir + "/template")

In [4]:
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
import pyspark.sql.types as T
import seaborn as sns
from pyspark.sql import Row, SparkSession
from pyspark.sql.window import Window

from config import load_config
from dataloader import load_line_info, load_test_data
from query_engine import GageClinet, SparkSessionBuilder

In [5]:
factory = "gumi"
topcodes = ["TOP41", "TOP42", "others"] # 데이터 로딩에 필요한 설정
model_number = 5 # 대상 모델 수(생산량 많은 순)

date_ranges = ("20250705", "20250810")
duration = {
   "start": "2025-07-07 07:00:00",
   "end": "2025-08-09 07:00:00"
}
# date_ranges = ("20250628", "20250706")
# duration = {
#    "start": "2025-06-30 07:00:00",
#    "end": "2025-07-05 07:00:00"
# }
# date_ranges = ("20250531", "20250706")
# duration = {
#    "start": "2025-06-02 07:00:00",
#    "end": "2025-07-05 07:00:00"
# }
# date_ranges = ("20250426", "20250706")
# duration = {
#    "start": "2025-04-28 07:00:00",
#    "end": "2025-07-05 07:00:00"
# }
# date_ranges = ("20250329", "20250706")
# duration = {
#    "start": "2025-03-31 07:00:00",
#    "end": "2025-07-05 07:00:00"
# }

In [6]:
config = load_config("config.toml", "spark", factory)
gage_cli = GageClinet(conf=config)
spark_sess = SparkSessionBuilder(conf=config)
spark_sess.show_spark_ui()

25/08/12 18:50:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


==check sparkui is up==
http://10.198.64.222:30011/user/shyeon.kang/proxy/4040/jobs/


In [7]:
master_df = load_test_data("master", date_ranges, topcodes, config, gage_cli, spark_sess)

found 1031 object files in an object storage


25/08/12 18:50:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


created pyspark dataframe with 1031 objects


In [8]:
# master_df.count()

## 데이터 파악
### 라인 정보 불러오기

In [9]:
# 사용 컬럼 
columns = [
   "dt",
   F.to_timestamp('INSP_DT', 'yyyyMMddHHmmss').alias('INSP_DT'),
   F.unix_timestamp("INSP_DT", 'yyyyMMddHHmmss').alias("UNIX_INSP_DT"),
   "P_N",
   "MODEL",
   "LINE_CODE",
   "BCR_IP",
   "JIG",
   "TEST_NAME",
   F.upper('RESULT').alias("RESULT"),
   "FAILITEM",
   "TESTCODE",
   "ERROR_CODE",
   F.col("TEST_TIME").cast(T.DoubleType())
   # "CN",
   # "IMEINO",
]

In [10]:
# 기준 정보
all_line_info = load_line_info(factory, spark_sess, ltype=None, maker=None)
all_line_info.show()

+-------+---------+----------------+------------+----------------+----------------+---------+----------+------+
|FACTORY|LINE_CODE|      LINE_NAME|PROCESS_TYPE|     MANUF_PART|       LINE_LOC|LINE_TYPE|OP_ST_TIME|IN_USE|
+-------+---------+----------------+------------+----------------+----------------+---------+----------+------+
|  GUMI|    M974|     A_BCELL_01|       MAIN|      MAIN(ATO)|     A_BCELL_01| CELL라인| 07:00:00|    N|
|  GUMI|    MA2V| B_B1_STORAGE_4|       MAIN|   MAIN STORAGE| B_B1_STORAGE_4| CELL라인| 08:00:00|    Y|
|  GUMI|    M011|   C_PCELL_2001|       MAIN|           MAIN|   PCELL_IC2001| CELL라인| 07:00:00|    N|
|  GUMI|    MA09|   ENGR_MAIN_01|       MAIN|     MAIN(ENGR)|   ENGR_MAIN_01| CELL라인| 07:00:00|    N|
|  GUMI|    MA0V|  INSPECTION 01|       MAIN|BLOCK INSPECTION|  INSPECTION 01|   BLOCK| 08:00:00|    Y|
|  GUMI|    M109|        MAIN 01|       MAIN|           MAIN|        MAIN 01| CELL라인| 07:00:00|    N|
|  GUMI|    MA2N|XR INSPECTION 01|       MAIN| XRD IN

### 모델, 공정별 수량 확인
- 해당 기간 중 생산량이 많은 모델을 자동 추출함

In [11]:
def null_condition(field_name: str):
   return (F.col(field_name) == "") | (F.col(field_name) == "null") | F.col(field_name).isNull()

In [12]:
basic_df = (
   master_df.select(columns)
   .where(~null_condition("P_N"))
   .where((F.col("INSP_DT") >= duration["start"]) & (F.col("INSP_DT") <= duration["end"]))
   # .where(F.col("MODEL").isin(models))
   # .where((F.col("ERROR_CODE") == "E034") & (F.col("TESTCODE").isin(["TOP50"])))
)

basic_df.printSchema()

root
 |-- dt: integer (nullable = true)
 |-- INSP_DT: timestamp (nullable = true)
 |-- UNIX_INSP_DT: long (nullable = true)
 |-- P_N: string (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- LINE_CODE: string (nullable = true)
 |-- BCR_IP: string (nullable = true)
 |-- JIG: string (nullable = true)
 |-- TEST_NAME: string (nullable = true)
 |-- RESULT: string (nullable = true)
 |-- FAILITEM: string (nullable = true)
 |-- TESTCODE: string (nullable = true)
 |-- ERROR_CODE: string (nullable = true)
 |-- TEST_TIME: double (nullable = true)



- 모델별 검사 투입량(생산량 추정)

In [13]:
production_num_pd = (
   basic_df
   .where(F.col("TESTCODE") == "TOP42")       
   .groupby("MODEL")
   .agg(
       F.count("*").alias("final_test_number")
   )
   .orderBy(F.desc("final_test_number"))
).toPandas()

models = production_num_pd.loc[:model_number-1, "MODEL"].tolist()
print("The best producting Models:", models)

The best producting Models: ['SM-F966N', 'SM-F766N', 'SM-S938B', 'SM-S931B', 'SM-S931N']


- 공정별 검사 투입량

In [14]:
# (
#    basic_df
#    .where(F.col("MODEL").isin(models))
#    .groupby("MODEL", "TEST_NAME", "TESTCODE")
#    .agg(
#        F.count("*").alias("test_number")
#    )
#    .orderBy("MODEL", F.desc("test_number"))
# ).show(300)

- Error Code 확인
   - E034 : Retest
   - E033 : 3회 이상 불량
   - E035 : MES INPUT 오류

In [15]:
# basic_df.select("ERROR_CODE").distinct().show()

## Retry, Retest 현황 분석

### Retry 대상 공정 파악
- E034가 남는 공정은 Retry 대상 공정으로 볼 수 있음

In [16]:
have_retry_df = (
   basic_df.where(
       F.col("MODEL").isin(models)
   ) # 해당 기간 중 생산량이 가장 많은 상위 모델
   .where(F.col("ERROR_CODE") == "E034")
   .select("MODEL", "TESTCODE")
   .distinct()
)

### 검사 프로세스 순서 파악
- 모델별 불량이 없고, 모든 공정의 이력이 한번씩만 있는 제품의 검사 이력을 정렬했을 때 가장 많은 순서 정보를 활용함

In [17]:
# PASS record만 사용
pass_df = (
   basic_df
   .where(F.col("MODEL").isin(models)) # 해당 기간 중 생산량이 가장 많은 상위 모델 
   .join(have_retry_df, ["MODEL", "TESTCODE"], "inner") # Retry 이력이 있음
   .where(F.col("RESULT") == "PASS")
)

In [18]:
# MODEL별 검사 공정이 일반적으로 몇 개인지 확인
wind_model_seq = (
   Window.partitionBy(["MODEL"])
   .orderBy(F.desc("count"))
)

testcode_dist_num_df = ( 
   pass_df.groupBy("MODEL", "P_N").agg(
       F.count_distinct("TESTCODE").alias("MODEL_TEST_DIST_NUM")
   )
   .groupBy("MODEL", "MODEL_TEST_DIST_NUM")
   .count()
   .where(F.col("MODEL_TEST_DIST_NUM") > 5) # 일부 공장에서만 사용하는 PN이 있어 제거하기 위함 (E034 없는 공정 중)
   .orderBy("MODEL", F.desc("count"))
   .withColumn("RANK", F.rank().over(wind_model_seq))
   .orderBy("MODEL", "RANK")
)

# testcode_dist_num_df.show(1000)

In [19]:
testcode_dist_num_df = (
   testcode_dist_num_df
   .where(F.col("RANK") == 1)
   .cache()   
)

In [20]:
# 모든 검사를 Pass한 세트의 검사 공정 기록을 시간 순으로 나열하여 순서를 매김
wind_pn_seq = Window.partitionBy(["MODEL", "P_N"]).orderBy(F.asc("INSP_DT"))
wind_pn_all = Window.partitionBy(["MODEL", "P_N"]).rowsBetween(
   Window.unboundedPreceding, Window.unboundedFollowing
)

pure_pass_set_df = (
   pass_df
   .withColumn(
       "PN_TEST_NUM", F.size(F.collect_list("TESTCODE").over(wind_pn_all))
   )
   .withColumn("PN_TEST_DIST_NUM", F.size(F.collect_set("TESTCODE").over(wind_pn_all)))
   .join(
       testcode_dist_num_df.select(["MODEL", "MODEL_TEST_DIST_NUM"]), "MODEL", "left" 
   ) 
   .where(
       (F.col("PN_TEST_NUM") == F.col("MODEL_TEST_DIST_NUM"))
       & (F.col("PN_TEST_DIST_NUM") == F.col("MODEL_TEST_DIST_NUM"))
   )
   .withColumn("TESTCODE_SEQUENCE", F.row_number().over(wind_pn_seq))
)

In [21]:
wind_spec = Window.partitionBy("MODEL", "TESTCODE_SEQUENCE").orderBy(
   F.desc("SEQUENCE_COUNT")
)
test_seq_df = (
   pure_pass_set_df.groupBy("MODEL", "TESTCODE_SEQUENCE", "TESTCODE")
   .agg(
       F.count("*").alias("SEQUENCE_COUNT"),
   )
   .withColumn("RANK", F.row_number().over(wind_spec))
)

# test_seq_df.show(1000)
# * 데이터 기간이 길어질수록 성능 문제가 있을 수 있으므로 모든 데이터를 활용할지는 추후 검토 필요

In [22]:
test_seq_df = (
   test_seq_df.where(F.col("RANK") == 1)
   .select(
       "MODEL",
       "TESTCODE",
       F.col("TESTCODE_SEQUENCE").alias("TEST_SEQ"),
   )
   .orderBy("MODEL", "TEST_SEQ")
)

test_seq_df.cache()
# test_seq_df.show(1000)

DataFrame[MODEL: string, TESTCODE: string, TEST_SEQ: int]

### Retry, Retest 현황 파악
- Retry Skip 조건: Retry 중에 동일 불량으로 배출 되는 경우
- Retest Skip 조건: Retest 를 포함하여 동일 불량으로 배출되는 경우

#### 연동 공정 (E034가 있는 공정 기준)
- TOP42, TOP31
- TOP54, TOP51

In [23]:
# 연동 공정을 감안해서 공정 순서 변경하기
adusted_seq_df = (
   basic_df
   .join(F.broadcast(test_seq_df), ["MODEL", "TESTCODE"], "inner")
   .withColumn(
       "INTER_TEST_SEQ",
       F.when(F.col("TESTCODE").isin(["TOP31", "TOP51"]), 1) # 연동 공정 표시 
       .otherwise(0),
   )
   .withColumn("TEST_SEQ", F.col("TEST_SEQ") - F.col("INTER_TEST_SEQ")) # 연동 공정은 순서가 같도록 조절    
)

In [24]:
# 리테스트 인덱스 붙이기 
wind_pn_seq = Window.partitionBy(["MODEL", "P_N"]).orderBy(F.asc("INSP_DT"))
wind_pn_all = (
   Window.partitionBy(["MODEL", "P_N"])
   .orderBy(F.asc("INSP_DT"))
   .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
)

retest_point = (
   (F.col("INSP_DT_DIFF") > 60 * 60 * 1) # 이전 검사와 1시간 이상 차이
   & (F.col("PRE_TEST_SEQ") >= F.col("TEST_SEQ")) # 검사 순서가 같거나 역행
)

added_retestid_df = (
   adusted_seq_df.withColumn(
       "PRE_TEST_SEQ", F.lag(F.col("TEST_SEQ"), 1).over(wind_pn_seq)
   )
   .fillna(0, subset=["PRE_TEST_SEQ"])
   .withColumn(
       "INSP_DT_DIFF",
       F.col("UNIX_INSP_DT") - F.lag(F.col("UNIX_INSP_DT"), 1).over(wind_pn_seq),
   )
   .withColumn(
       "RETEST_GID",
       F.sum(F.when(retest_point, 1).otherwise(0)).over(wind_pn_seq),
   )
   # 검증을 위한 로직
   # .withColumn("RETEST_GIDS", F.collect_list("RETEST_GID").over(wind_pn_all))
   # .where(F.array_contains("RETEST_GIDS", 1))
)

# added_retestid_df.show(100)

In [25]:
# Retry 인덱스 붙이기 
wind_cols = ["MODEL", "P_N", "TEST_SEQ", "TESTCODE", "RETEST_GID"]
wind_seq = Window.partitionBy(wind_cols).orderBy(F.asc("INSP_DT"))
wind_all = (
   Window.partitionBy(wind_cols)
   .orderBy(F.asc("INSP_DT"))
   .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
)

is_preerrorcode_null = (
   (F.col("PRE_ERROR_CODE") == "null")
   | (F.col("PRE_ERROR_CODE") == "")
   | (F.col("PRE_ERROR_CODE").isNull())
)
retry_point = (
   (F.col("ERROR_CODE") == "E034")
   & is_preerrorcode_null
   & (F.col("PRE_TEST_SEQ") == F.col("TEST_SEQ"))
)


added_retryid_df = (
   added_retestid_df
   .withColumn(
       "PRE_ERROR_CODE", F.lag(F.col("ERROR_CODE"), 1).over(wind_seq)
   )
   .withColumn("ERROR_CODES", F.collect_list(F.col("ERROR_CODE")).over(wind_all))
   .withColumn("PRE_TEST_SEQ", F.lag(F.col("TEST_SEQ")).over(wind_seq))
   .withColumn(
       "RETRY_GID",
       F.sum(F.when(retry_point, 1).otherwise(0)).over(wind_seq),
   )
   # 검증을 위한 로직
   # .withColumn("RETEST_GIDS", F.collect_list("RETEST_GID").over(wind_pn_all))
   # .withColumn("RETRY_GIDS", F.collect_list("RETRY_GID").over(wind_pn_all)) 
   # .where(F.array_contains("RETRY_GIDS", 1) | F.array_contains("RETEST_GIDS", 1))   
)

columns = [
   "INSP_DT",
   "P_N",
   "MODEL",
   F.concat_ws("_", F.col("BCR_IP"), F.col("JIG")).alias("IPJIG"),
   "TESTCODE",
   "TEST_NAME",
   "PRE_TEST_SEQ",
   "TEST_SEQ",   
   "RETEST_GID",   
   "RETRY_GID",   
   "RESULT",
   "PRE_ERROR_CODE",
   "ERROR_CODE",
   "FAILITEM",
   "TEST_TIME"
]

retest_base_df = (
   added_retryid_df.select(columns)
)

# 검증 테이블
# retest_base_df.orderBy("MODEL", "P_N", "INSP_DT").show(100, truncate=False)

#### Retry 현황

In [26]:
wind_cols = ["P_N", "TESTCODE", "RETEST_GID", "RETRY_GID"]
wind_seq = Window.partitionBy(wind_cols).orderBy(F.asc("INSP_DT"))
wind_all = (
   Window.partitionBy(wind_cols)
   .orderBy(F.asc("INSP_DT"))
   .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
)

retry_analytic_df = (
   retest_base_df.withColumn("RESULTS", F.collect_list("RESULT").over(wind_all))
   .withColumn("DIST_RESULTS", F.collect_set("RESULT").over(wind_all))
   .withColumn("DIST_IPJIG", F.collect_set("IPJIG").over(wind_all))
   .withColumn("SEQ", F.row_number().over(wind_seq))  # 첫 번째 결과만 추후 활용하기 위함
   .withColumn(
       "IS_TRUE_ALRAM",
       F.array_contains("DIST_RESULTS", "FAIL") & (F.size("DIST_RESULTS") == 1) & (F.size("RESULTS") > 1),
   )
   .withColumn(
       "IS_FALSE_ALRAM",
       F.array_contains("RESULTS", "PASS") & F.array_contains("RESULTS", "FAIL"),
   )
   .withColumn(
       "IS_NO_ALRAM",
       F.array_contains("DIST_RESULTS", "PASS") & (F.size("DIST_RESULTS") == 1),
   )
   .withColumn(
       "IS_CHANGED_IPJIG", F.when((F.size("DIST_IPJIG") > 1), True).otherwise(False)
   )
   .withColumn("FIRST_FAILITEM", F.first("FAILITEM").over(wind_all))
   .withColumn(
       "FIRST_FAILITEM",
       F.when(F.col("IS_NO_ALRAM") == True, F.col("TESTCODE")).otherwise(
           F.col("FIRST_FAILITEM")
       ), # 양품이었던 검사는 검사 항목을 남겨서 추후 양품 수를 추정하는 데 사용
   )
   .where(F.col("IS_TRUE_ALRAM") | F.col("IS_FALSE_ALRAM") | F.col("IS_NO_ALRAM")) # 3개 조건 중 하나는 true
   .withColumn("TEST_NUM", F.size("RESULTS")) # 반복 검사 시 횟수를 남김   
   .withColumn("TEST_TIME_SUM", (F.sum("TEST_TIME").over(wind_all))) # 조건별 검사 시간 확인   
)

columns = [
   "INSP_DT",
   "P_N",
   "MODEL",
   "TEST_NAME",
   "RETRY_GID",
   "RETEST_GID",
   "RESULTS",
   "TEST_NUM",
   "FIRST_FAILITEM",
   "IS_TRUE_ALRAM",
   "IS_FALSE_ALRAM",
   "IS_NO_ALRAM",
   "IS_CHANGED_IPJIG",
   "SEQ",
   "TESTCODE",
   "TEST_TIME_SUM"
]

retry_analytic_df = (
   retry_analytic_df.select(columns)
   # 검증용
)#.orderBy("MODEL", "P_N", "TEST_SEQ", "INSP_DT").show(1000, truncate=False)

In [27]:
retry_analytic_pd = (
   retry_analytic_df.where(F.col("SEQ") == 1)
   .groupBy("MODEL", "TEST_NAME", "TESTCODE", "FIRST_FAILITEM")
   .agg(
       F.sum(F.when(F.col("IS_TRUE_ALRAM") == "true", 1).otherwise(0)).alias("TRUE_ALRAM_INPUTNUM"),
       F.sum(F.when(F.col("IS_FALSE_ALRAM") == "true", 1).otherwise(0)).alias("FALSE_ALRAM_INPUTNUM"),
       F.sum(F.when(F.col("IS_NO_ALRAM") == "true", 1).otherwise(0)).alias("NO_ALRAM_INPUTNUM"),
       F.sum(F.when(F.col("IS_TRUE_ALRAM") == "true", F.col("TEST_NUM")).otherwise(0)).alias("TRUE_ALRAM_TESTNUM"),
       F.sum(F.when(F.col("IS_FALSE_ALRAM") == "true", F.col("TEST_NUM")).otherwise(0)).alias("FALSE_ALRAM_TESTNUM"),
       F.sum(F.when(F.col("IS_NO_ALRAM") == "true", F.col("TEST_NUM")).otherwise(0)).alias("NO_ALRAM_TESTNUM"),
       F.sum(F.when(F.col("IS_TRUE_ALRAM") == "true", F.col("TEST_TIME_SUM")).otherwise(0)).alias("TRUE_ALRAM_TESTTIME_SUM"),
       F.sum(F.when(F.col("IS_FALSE_ALRAM") == "true", F.col("TEST_TIME_SUM")).otherwise(0)).alias("FALSE_ALRAM_TESTTIME_SUM"),
       F.sum(F.when(F.col("IS_NO_ALRAM") == "true", F.col("TEST_TIME_SUM")).otherwise(0)).alias("NO_ALRAM_TESTTIME_SUM"),
       F.sum(F.when(F.col("IS_CHANGED_IPJIG") == "true", 1).otherwise(0)).alias("CHANGED_IPJIG_SUM"),
   )
   .withColumn("TRUE_ALRAM_TESTTIME_AVG", F.round(F.col("TRUE_ALRAM_TESTTIME_SUM") / F.col("TRUE_ALRAM_TESTNUM"), 2))
   .withColumn("FALSE_ALRAM_TESTTIME_AVG", F.round(F.col("FALSE_ALRAM_TESTTIME_SUM") / F.col("FALSE_ALRAM_TESTNUM"), 2))   
   .withColumn("NO_ALRAM_TESTTIME_AVG", F.round(F.col("NO_ALRAM_TESTTIME_SUM") / F.col("NO_ALRAM_TESTNUM"), 2))
   .withColumn("NO_ALRAM_TESTTIME_AVG", F.round(F.col("NO_ALRAM_TESTTIME_SUM") / F.col("NO_ALRAM_TESTNUM"), 2))
   .withColumn("TRUE_ALRAM_RATE", F.round(F.col("TRUE_ALRAM_INPUTNUM") / (F.col("TRUE_ALRAM_INPUTNUM") + F.col("FALSE_ALRAM_INPUTNUM")), 2))
).toPandas().fillna(0)

retry_analytic_pd.head(100)

25/08/12 18:57:11 WARN DAGScheduler: Broadcasting large task binary with size 1858.3 KiB
25/08/12 18:57:48 WARN DAGScheduler: Broadcasting large task binary with size 2041.7 KiB
25/08/12 18:58:34 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
25/08/12 18:59:21 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
25/08/12 19:00:01 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
25/08/12 19:00:42 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
25/08/12 19:01:29 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/08/12 19:01:46 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/08/12 19:02:03 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


,MODEL,TEST_NAME,TESTCODE,FIRST_FAILITEM,TRUE_ALRAM_INPUTNUM,FALSE_ALRAM_INPUTNUM,NO_ALRAM_INPUTNUM,TRUE_ALRAM_TESTNUM,FALSE_ALRAM_TESTNUM,NO_ALRAM_TESTNUM,TRUE_ALRAM_TESTTIME_SUM,FALSE_ALRAM_TESTTIME_SUM,NO_ALRAM_TESTTIME_SUM,CHANGED_IPJIG_SUM,TRUE_ALRAM_TESTTIME_AVG,FALSE_ALRAM_TESTTIME_AVG,NO_ALRAM_TESTTIME_AVG,TRUE_ALRAM_RATE
0,SM-S931N,AUDIO F/T,TOP26,SPK AMP[340],1,6,0,2,13,0,55.6,454.9,0.0,4,27.80,34.99,0.0,0.14
1,SM-F766N,CAMERA MAIN F/T,TOPD33,FW1_Camera Capture,22,132,0,61,280,0,2295.0,9531.0,0.0,87,37.62,34.04,0.0,0.14
2,SM-S931B,FINAL LTE,TOP42,NR_n77_RX_DL623330CH UL623330CH_S638 R15 A32 P...,35,6,0,88,16,0,20003.0,3193.0,0.0,32,227.31,199.56,0.0,0.85
3,SM-F966N,RF CAL LTE,TOP41,Execute Prevent Process Secession,18,17,0,45,43,0,3176.0,5439.0,0.0,33,70.58,126.49,0.0,0.51
4,SM-F966N,RF CAL LTE,TOP41,NR_n77B_Tx_S821 R23 A22 P8 N6_50MHz XPT Cal St...,5,47,0,11,111,0,2929.0,29548.0,0.0,48,266.27,266.20,0.0,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SM-F966N,RF CAL LTE,TOP41,Vibrator Self Test_X,7,2,0,15,4,0,2687.0,1048.0,0.0,5,179.13,262.00,0.0,0.78
96,SM-S931B,RF CAL_CP,TOP32,FW1_CAM_ON_TEST(Mode : Full),31,46,0,63,133,0,6706.0,13320.0,0.0,65,106.44,100.15,0.0,0.40
97,SM-F766N,IMEI W&C,TOP10,Set USB to MTP Fail[E35-216],0,114,0,0,230,0,0.0,120415.0,0.0,114,0.00,523.54,0.0,0.00
98,SM-F966N,RF CAL LTE,TOP41,NR_n3_Tx_S222 R23 A8 P8 N4_30MHz XPT Cal Status,0,10,0,0,22,0,0.0,6251.0,0.0,6,0.00,284.14,0.0,0.00


In [28]:
pass_df = retry_analytic_pd[retry_analytic_pd["FIRST_FAILITEM"] == retry_analytic_pd["TESTCODE"].astype(str)]
pass_cols = ["MODEL", "TESTCODE"] + [col for col in retry_analytic_pd.columns if col[:len("NO_ALRAM_")] == "NO_ALRAM_"]
fail_df = retry_analytic_pd[retry_analytic_pd["FIRST_FAILITEM"] != retry_analytic_pd["TESTCODE"].astype(str)]
fail_cols = [col for col in retry_analytic_pd.columns if col[:len("NO_ALRAM_")] != "NO_ALRAM_"]

result_pd = fail_df[fail_cols].merge(pass_df[pass_cols], on=["MODEL", "TESTCODE"], how="left")
result_pd.head()

,MODEL,TEST_NAME,TESTCODE,FIRST_FAILITEM,TRUE_ALRAM_INPUTNUM,FALSE_ALRAM_INPUTNUM,TRUE_ALRAM_TESTNUM,FALSE_ALRAM_TESTNUM,TRUE_ALRAM_TESTTIME_SUM,FALSE_ALRAM_TESTTIME_SUM,CHANGED_IPJIG_SUM,TRUE_ALRAM_TESTTIME_AVG,FALSE_ALRAM_TESTTIME_AVG,TRUE_ALRAM_RATE,NO_ALRAM_INPUTNUM,NO_ALRAM_TESTNUM,NO_ALRAM_TESTTIME_SUM,NO_ALRAM_TESTTIME_AVG
0,SM-S931N,AUDIO F/T,TOP26,SPK AMP[340],1,6,2,13,55.6,454.9,4,27.80,34.99,0.14,108305,108332,2185214.7,20.17
1,SM-F766N,CAMERA MAIN F/T,TOPD33,FW1_Camera Capture,22,132,61,280,2295.0,9531.0,87,37.62,34.04,0.14,237995,238886,8191268.0,34.29
2,SM-S931B,FINAL LTE,TOP42,NR_n77_RX_DL623330CH UL623330CH_S638 R15 A32 P...,35,6,88,16,20003.0,3193.0,32,227.31,199.56,0.85,128002,128853,20470417.0,158.87
3,SM-F966N,RF CAL LTE,TOP41,Execute Prevent Process Secession,18,17,45,43,3176.0,5439.0,33,70.58,126.49,0.51,227440,227613,48139756.0,211.50
4,SM-F966N,RF CAL LTE,TOP41,NR_n77B_Tx_S821 R23 A22 P8 N6_50MHz XPT Cal St...,5,47,11,111,2929.0,29548.0,48,266.27,266.20,0.10,227440,227613,48139756.0,211.50


In [29]:
result_pd.sort_values(by=["MODEL", "TEST_NAME", "TRUE_ALRAM_RATE"], ascending=False).to_csv(f"retry_{factory}_{duration['start']}_{duration['end']}.csv")

#### Retest 현황

In [30]:
wind_cols = ["P_N", "TESTCODE", "RETEST_GID"]
wind_seq = Window.partitionBy(wind_cols).orderBy(F.asc("INSP_DT"))
wind_all = (
   Window.partitionBy(wind_cols)
   .orderBy(F.asc("INSP_DT"))
   .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
)

retest_analytic_df = (
   retest_base_df.withColumn("RESULTS", F.collect_list("RESULT").over(wind_all))
   .withColumn("DIST_RESULTS", F.collect_set("RESULT").over(wind_all))
   .withColumn("DIST_IPJIG", F.collect_set("IPJIG").over(wind_all))   
   .withColumn("SEQ", F.row_number().over(wind_seq))   
   .withColumn(
       "IS_TRUE_ALRAM",
       F.array_contains("DIST_RESULTS", "FAIL") & (F.size("DIST_RESULTS") == 1) & (F.size("RESULTS") > 1),
   )
   .withColumn(
       "IS_FALSE_ALRAM",
       F.array_contains("RESULTS", "PASS") & F.array_contains("RESULTS", "FAIL"),
   )
   .withColumn(
       "IS_NO_ALRAM",
       F.array_contains("DIST_RESULTS", "PASS") & (F.size("DIST_RESULTS") == 1),
   )
   .withColumn(
       "IS_CHANGED_IPJIG", F.when((F.size("DIST_IPJIG") > 1), True).otherwise(False)
   )   
   .withColumn("FIRST_FAILITEM", F.first("FAILITEM").over(wind_all))
   .withColumn(
       "FIRST_FAILITEM",
       F.when(F.col("IS_NO_ALRAM") == True, F.col("TESTCODE")).otherwise(
           F.col("FIRST_FAILITEM")
       ), # 양품이었던 검사는 검사 항목을 남겨서 추후 양품 수를 추정하는 데 사용
   )
   .where(F.col("IS_TRUE_ALRAM") | F.col("IS_FALSE_ALRAM") | F.col("IS_NO_ALRAM")) # 3개 조건 중 하나는 true
   .withColumn("TEST_NUM", F.size("RESULTS")) # 반복 검사 시 횟수를 남김   
   .withColumn("TEST_TIME_SUM", (F.sum("TEST_TIME").over(wind_all))) # 조건별 검사 시간 확인   
)

columns = [
   "INSP_DT",
   "P_N",
   "MODEL",
   "TEST_NAME",
   "RETRY_GID",
   "RETEST_GID",
   "RESULTS",
   "TEST_NUM",
   "FIRST_FAILITEM",
   "IS_TRUE_ALRAM",
   "IS_FALSE_ALRAM",
   "IS_NO_ALRAM",
   "IS_CHANGED_IPJIG",   
   "SEQ",
   "TESTCODE",
   "TEST_TIME_SUM"
]

retest_analytic_df = (
   retest_analytic_df.select(columns)
   # 검증용
)#.orderBy("MODEL", "P_N", "TEST_SEQ", "INSP_DT").show(1000, truncate=False)

In [31]:
retest_analytic_pd = (
   retest_analytic_df.where(F.col("SEQ") == 1)
   .groupBy("MODEL", "TEST_NAME", "TESTCODE", "FIRST_FAILITEM")
   .agg(
       F.sum(F.when(F.col("IS_TRUE_ALRAM") == "true", 1).otherwise(0)).alias("TRUE_ALRAM_INPUTNUM"),
       F.sum(F.when(F.col("IS_FALSE_ALRAM") == "true", 1).otherwise(0)).alias("FALSE_ALRAM_INPUTNUM"),
       F.sum(F.when(F.col("IS_NO_ALRAM") == "true", 1).otherwise(0)).alias("NO_ALRAM_INPUTNUM"),
       F.sum(F.when(F.col("IS_TRUE_ALRAM") == "true", F.col("TEST_NUM")).otherwise(0)).alias("TRUE_ALRAM_TESTNUM"),
       F.sum(F.when(F.col("IS_FALSE_ALRAM") == "true", F.col("TEST_NUM")).otherwise(0)).alias("FALSE_ALRAM_TESTNUM"),
       F.sum(F.when(F.col("IS_NO_ALRAM") == "true", F.col("TEST_NUM")).otherwise(0)).alias("NO_ALRAM_TESTNUM"),
       F.sum(F.when(F.col("IS_TRUE_ALRAM") == "true", F.col("TEST_TIME_SUM")).otherwise(0)).alias("TRUE_ALRAM_TESTTIME_SUM"),
       F.sum(F.when(F.col("IS_FALSE_ALRAM") == "true", F.col("TEST_TIME_SUM")).otherwise(0)).alias("FALSE_ALRAM_TESTTIME_SUM"),
       F.sum(F.when(F.col("IS_NO_ALRAM") == "true", F.col("TEST_TIME_SUM")).otherwise(0)).alias("NO_ALRAM_TESTTIME_SUM"),
       F.sum(F.when(F.col("IS_CHANGED_IPJIG") == "true", 1).otherwise(0)).alias("CHANGED_IPJIG_SUM"),       
   )
   .withColumn("TRUE_ALRAM_TESTTIME_AVG", F.round(F.col("TRUE_ALRAM_TESTTIME_SUM") / F.col("TRUE_ALRAM_TESTNUM"), 2))
   .withColumn("FALSE_ALRAM_TESTTIME_AVG", F.round(F.col("FALSE_ALRAM_TESTTIME_SUM") / F.col("FALSE_ALRAM_TESTNUM"), 2))   
   .withColumn("NO_ALRAM_TESTTIME_AVG", F.round(F.col("NO_ALRAM_TESTTIME_SUM") / F.col("NO_ALRAM_TESTNUM"), 2))
   .withColumn("NO_ALRAM_TESTTIME_AVG", F.round(F.col("NO_ALRAM_TESTTIME_SUM") / F.col("NO_ALRAM_TESTNUM"), 2))
   .withColumn("TRUE_ALRAM_RATE", F.round(F.col("TRUE_ALRAM_INPUTNUM") / (F.col("TRUE_ALRAM_INPUTNUM") + F.col("FALSE_ALRAM_INPUTNUM")), 2))
).toPandas().fillna(0)

retest_analytic_pd.head(100)

25/08/12 19:02:05 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
25/08/12 19:02:59 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/08/12 19:03:10 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/08/12 19:03:25 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


,MODEL,TEST_NAME,TESTCODE,FIRST_FAILITEM,TRUE_ALRAM_INPUTNUM,FALSE_ALRAM_INPUTNUM,NO_ALRAM_INPUTNUM,TRUE_ALRAM_TESTNUM,FALSE_ALRAM_TESTNUM,NO_ALRAM_TESTNUM,TRUE_ALRAM_TESTTIME_SUM,FALSE_ALRAM_TESTTIME_SUM,NO_ALRAM_TESTTIME_SUM,CHANGED_IPJIG_SUM,TRUE_ALRAM_TESTTIME_AVG,FALSE_ALRAM_TESTTIME_AVG,NO_ALRAM_TESTTIME_AVG,TRUE_ALRAM_RATE
0,SM-S931N,IMEI W&C,TOP10,Modem Port Search Fail(Odin DLL - USB Disconne...,0,2,0,0,4,0,0.0,2847.0,0.0,2,0.00,711.75,0.0,0.00
1,SM-S931N,AUDIO F/T,TOP26,SPK AMP[340],1,6,0,2,13,0,55.6,454.9,0.0,4,27.80,34.99,0.0,0.14
2,SM-S931B,W-LAN,TOP31,11n_2.4G_Packet Count Tx Measure Error,1,5,0,2,10,0,507.0,1291.0,0.0,5,253.50,129.10,0.0,0.17
3,SM-S931N,AUDIO F/T,TOP26,SPK2 AMP[205],0,4,0,0,8,0,0.0,185.4,0.0,2,0.00,23.17,0.0,0.00
4,SM-F766N,CAMERA MAIN F/T,TOPD33,FW1_Camera Capture,22,130,0,61,276,0,2295.0,9381.0,0.0,87,37.62,33.99,0.0,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SM-S931N,FINAL LTE,TOP42,NR_n78_RX_DL649998CH UL649998CH_S638 R15 A32 P...,1,1,0,6,2,0,1069.0,321.0,0.0,2,178.17,160.50,0.0,0.50
96,SM-F966N,PD F/T,TOPD67,DC Init Status Result Fail,0,19,0,0,39,0,0.0,4517.3,0.0,4,0.00,115.83,0.0,0.00
97,SM-S938B,FINAL LTE,TOP42,WCDMA Freq. Error,1,0,0,4,0,0,1118.0,0.0,0.0,1,279.50,0.00,0.0,1.00
98,SM-F766N,FINAL LTE,TOP42,NR_n78_RX_DL620666CH UL620666CH_PRX MIMO_RX 3r...,63,8,0,302,34,0,35864.0,4284.0,0.0,69,118.75,126.00,0.0,0.89


In [32]:
pass_df = retest_analytic_pd[retest_analytic_pd["FIRST_FAILITEM"] == retest_analytic_pd["TESTCODE"].astype(str)]
pass_cols = ["MODEL", "TESTCODE"] + [col for col in retest_analytic_pd.columns if col[:len("NO_ALRAM_")] == "NO_ALRAM_"]
fail_df = retest_analytic_pd[retest_analytic_pd["FIRST_FAILITEM"] != retest_analytic_pd["TESTCODE"].astype(str)]
fail_cols = [col for col in retest_analytic_pd.columns if col[:len("NO_ALRAM_")] != "NO_ALRAM_"]

result_pd = fail_df[fail_cols].merge(pass_df[pass_cols], on=["MODEL", "TESTCODE"], how="left")
result_pd.head()

,MODEL,TEST_NAME,TESTCODE,FIRST_FAILITEM,TRUE_ALRAM_INPUTNUM,FALSE_ALRAM_INPUTNUM,TRUE_ALRAM_TESTNUM,FALSE_ALRAM_TESTNUM,TRUE_ALRAM_TESTTIME_SUM,FALSE_ALRAM_TESTTIME_SUM,CHANGED_IPJIG_SUM,TRUE_ALRAM_TESTTIME_AVG,FALSE_ALRAM_TESTTIME_AVG,TRUE_ALRAM_RATE,NO_ALRAM_INPUTNUM,NO_ALRAM_TESTNUM,NO_ALRAM_TESTTIME_SUM,NO_ALRAM_TESTTIME_AVG
0,SM-S931N,IMEI W&C,TOP10,Modem Port Search Fail(Odin DLL - USB Disconne...,0,2,0,4,0.0,2847.0,2,0.00,711.75,0.00,108809,108877,43624317.0,400.68
1,SM-S931N,AUDIO F/T,TOP26,SPK AMP[340],1,6,2,13,55.6,454.9,4,27.80,34.99,0.14,108305,108332,2185214.7,20.17
2,SM-S931B,W-LAN,TOP31,11n_2.4G_Packet Count Tx Measure Error,1,5,2,10,507.0,1291.0,5,253.50,129.10,0.17,129298,129725,5543857.0,42.74
3,SM-S931N,AUDIO F/T,TOP26,SPK2 AMP[205],0,4,0,8,0.0,185.4,2,0.00,23.17,0.00,108305,108332,2185214.7,20.17
4,SM-F766N,CAMERA MAIN F/T,TOPD33,FW1_Camera Capture,22,130,61,276,2295.0,9381.0,87,37.62,33.99,0.14,237979,238870,8190709.0,34.29


In [33]:
result_pd.to_csv(f"retest_{factory}_{duration['start']}_{duration['end']}.csv")

25/08/12 19:35:24 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
io.fabric8.kubernetes.client.WatcherException: too old resource version: 289357396 (289363876)
	at io.fabric8.kubernetes.client.dsl.internal.WatchConnectionManager$TypedWatcherWebSocketListener.onMessage(WatchConnectionManager.java:103)
	at okhttp3.internal.ws.RealWebSocket.onReadMessage(RealWebSocket.java:323)
	at okhttp3.internal.ws.WebSocketReader.readMessageFrame(WebSocketReader.java:219)
	at okhttp3.internal.ws.WebSocketReader.processNextFrame(WebSocketReader.java:105)
	at okhttp3.internal.ws.RealWebSocket.loopReader(RealWebSocket.java:274)
	at okhttp3.internal.ws.RealWebSocket$2.onResponse(RealWebSocket.java:214)
	at okhttp3.RealCall$AsyncCall.execute(RealCall.java:203)
	at okhttp3.internal.NamedRunnable.run(NamedRunnable.java:32)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base